In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
# import datetime

In [21]:
df = pd.read_pickle('cleaned.pickle')

In [22]:
df

,着順,枠,馬番,馬名,斤量,騎手,人気,性,齢,体重,体重変化
202001010101,1,6,6,ウインルーア,54.0,横山武,3.0,牝,2,438,4
202001010101,2,2,2,アークライト,54.0,ルメー,2.0,牡,2,510,0
202001010101,3,3,3,ギャラントウォリア,54.0,池添,1.0,牡,2,482,-6
202001010101,4,1,1,ジュンブーケ,52.0,△亀田,4.0,牝,2,442,0
202001010101,5,4,4,キタノマンゲツ,54.0,藤岡康,5.0,牡,2,426,-8
...,...,...,...,...,...,...,...,...,...,...,...
202002020612,12,6,12,ヴァラークラウン,52.0,古川,5.0,牝,3,468,-4
202002020612,13,2,4,ジゲン,54.0,池添,3.0,牡,3,440,-8
202002020612,14,4,7,テーオーアマゾン,57.0,水口,16.0,牡,4,486,4
202002020612,15,1,2,イルヴェントデーア,55.0,藤岡佑,7.0,牝,4,448,0


In [23]:
race_id_list = df.index.unique()
race_id_list

Index(['202001010101', '202001010102', '202001010103', '202001010104',
       '202001010105', '202001010106', '202001010107', '202001010108',
       '202001010109', '202001010110',
       ...
       '202002020603', '202002020604', '202002020605', '202002020606',
       '202002020607', '202002020608', '202002020609', '202002020610',
       '202002020611', '202002020612'],
      dtype='object', length=312)

In [24]:
race_infos = {}
for race_id in tqdm(race_id_list):
  url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
  html = requests.get(url)
  html.encoding= "EUC_JP"
  soup = BeautifulSoup(html.text, "html.parser")

  日付 = soup.find("dd",class_='Active')
  日付 = 日付.find("a").get_text()
  レース情報 = soup.find("div", class_="RaceList_Item02") 
  # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
  # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
  レース環境 = レース情報.find("div",class_="RaceData01").get_text()
  info = re.findall(r"\w+",レース環境)
  info[0] = info[2][:1]
  info[1] = info[2][1:]
  info.pop(2)
  info_dict = {}
  # print(info)
  info_dict["date"] = f"{race_id[:4]}年{日付[:-3]}"

  for text in info:
    # print(text)
    if text in ["芝","ダ"]:
      # print("芝付き")
      info_dict["race_type"] = text
    if "障" in text:
      info_dict["race_type"] = "障害"
    if "m" in text:
      # print("mつき")
      info_dict["course_ren"] = re.findall(r"\d+",text)[0]
    if text in ["雨","小雨","小雪","雪","晴","曇"]:
      # print("天気")
      info_dict["weather"] = text
    # if "年" in text:
    #   info_dict["data"] = text
    if text in ["良","不良","重","稍","不"]:
      # print("馬場")
      info_dict["ground_state"] = text


  race_infos[race_id] = info_dict
  time.sleep(.5)

100%|██████████| 312/312 [04:42<00:00,  1.11it/s]


In [25]:
race_infos_df = pd.DataFrame(race_infos).T
race_infos_df

,date,race_type,course_ren,weather,ground_state
202001010101,2020年7月25日,芝,1800,曇,良
202001010102,2020年7月25日,ダ,1700,曇,良
202001010103,2020年7月25日,芝,1500,曇,良
202001010104,2020年7月25日,ダ,1700,曇,良
202001010105,2020年7月25日,芝,1200,晴,良
...,...,...,...,...,...
202002020608,2020年7月19日,芝,1800,曇,良
202002020609,2020年7月19日,芝,1200,曇,良
202002020610,2020年7月19日,ダ,1700,曇,良
202002020611,2020年7月19日,芝,2000,曇,良


In [26]:
results_addinfo = df.merge(race_infos_df,left_index=True,right_index=True,how="inner")

In [27]:
results_addinfo["date"] = pd.to_datetime(results_addinfo["date"],format="%Y年%m月%d日")
results_addinfo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3685 entries, 202001010101 to 202002020612
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            3685 non-null   int64         
 1   枠             3685 non-null   int64         
 2   馬番            3685 non-null   int64         
 3   馬名            3685 non-null   object        
 4   斤量            3685 non-null   float64       
 5   騎手            3685 non-null   object        
 6   人気            3685 non-null   float64       
 7   性             3685 non-null   object        
 8   齢             3685 non-null   int64         
 9   体重            3685 non-null   int64         
 10  体重変化          3685 non-null   int64         
 11  date          3685 non-null   datetime64[ns]
 12  race_type     3685 non-null   object        
 13  course_ren    3685 non-null   object        
 14  weather       3685 non-null   object        
 15  ground_state  3685 non-n

In [28]:
results_addinfo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3685 entries, 202001010101 to 202002020612
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            3685 non-null   int64         
 1   枠             3685 non-null   int64         
 2   馬番            3685 non-null   int64         
 3   馬名            3685 non-null   object        
 4   斤量            3685 non-null   float64       
 5   騎手            3685 non-null   object        
 6   人気            3685 non-null   float64       
 7   性             3685 non-null   object        
 8   齢             3685 non-null   int64         
 9   体重            3685 non-null   int64         
 10  体重変化          3685 non-null   int64         
 11  date          3685 non-null   datetime64[ns]
 12  race_type     3685 non-null   object        
 13  course_ren    3685 non-null   object        
 14  weather       3685 non-null   object        
 15  ground_state  3685 non-n

In [29]:
len(results_addinfo)

3685

In [30]:
results_addinfo.to_pickle('df.pickle')